In [4]:
import os
import re
import uuid
import fitz
import json
from tqdm import tqdm
# Updated imports for newer LangChain versions
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import sys
from pathlib import Path

# ==========================
# CONFIGURATION
# ==========================

# Make the project root visible for imports
PROJECT_ROOT = Path().resolve().parent
sys.path.append(str(PROJECT_ROOT))

from config.paths import DATA_DIR , PROCESSED_DIR , VECTORSTORE_DIR


EMBEDDING_MODEL = "BAAI/bge-large-en-v1.5"



/home/jagannath/my_poc_law/MyPocketLawyer-AI-Powered-Legal-Aid-Assistant/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:

# ==========================
# TEXT PROCESSING HELPERS
# ==========================

def load_pdf_text(pdf_path: Path) -> str:
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text("text")
    return text

def clean_legal_text(text: str) -> str:
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'–', '-', text)
    return text.strip()

def segment_legal_sections(text: str) -> list:
    """
    Split legal PDFs into chapters/parts and sections.
    Handles:
      - Chapter/Part with numbers or Roman numerals
      - Multi-line chapter titles
      - Sections kept together
      - Optional clauses preserved but not split line-by-line
    """
    sections = []
    current_chapter = None
    current_section = None
    buffer = []

    lines = [line.strip() for line in text.split('\n') if line.strip()]
    i = 0
    while i < len(lines):
        line = lines[i]

        # Detect Chapter or Part headers
        chapter_match = re.match(r'^(Chapter|CHAPTER|Part|PART)[\s–-]*([IVXLC\d]+)?\s*(.*)', line)
        if chapter_match:
            # Flush previous buffer
            if buffer:
                sections.append({
                    "chapter": current_chapter or "Unspecified Chapter/Part",
                    "section": current_section,
                    "content": " ".join(buffer).strip()
                })
                buffer = []

            # Start new chapter
            current_chapter = f"{chapter_match.group(1)} {chapter_match.group(2) or ''}".strip()

            # Check next line for possible title
            if i + 1 < len(lines):
                next_line = lines[i + 1]
                # If it does NOT look like a section or clause, append as chapter title
                if not re.match(r'^(\d+[\.\)]\s*)', next_line):
                    current_chapter += f" {next_line}"
                    i += 1  # Skip the title line

            current_section = None
            i += 1
            continue

        # Detect numbered sections (e.g., '1.' or '10)')
        section_match = re.match(r'^(\d+[\.\)]\s*)(.*)', line)
        if section_match:
            # Flush previous buffer
            if buffer:
                sections.append({
                    "chapter": current_chapter or "Unspecified Chapter/Part",
                    "section": current_section,
                    "content": " ".join(buffer).strip()
                })
                buffer = []

            # Start new section
            current_section = line
            i += 1
            continue

        # Optional: detect clauses (1), (2) but keep in same section
        clause_match = re.findall(r'\(\d+\)\s*([^()]+)', line)
        if clause_match:
            for clause in clause_match:
                buffer.append(clause.strip())
            i += 1
            continue

        # Regular content line
        buffer.append(line)
        i += 1

    # Flush remaining buffer
    if buffer:
        sections.append({
            "chapter": current_chapter or "Unspecified Chapter/Part",
            "section": current_section,
            "content": " ".join(buffer).strip()
        })

    return sections


def save_json(data, path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)





In [6]:
# ==========================
# MAIN PIPELINE
# ==========================

def process_legal_pdfs(data_dir: Path, processed_dir: Path):
    all_docs = []
    pdf_files = list(data_dir.glob("*.pdf"))
    if not pdf_files:
        print(f"⚠️ No PDF files found in {data_dir}")
        return []

    for pdf_path in tqdm(pdf_files, desc="📄 Processing PDFs"):
        text = load_pdf_text(pdf_path)
        sections = segment_legal_sections(text)

        for s in sections:
            s["doc_id"] = str(uuid.uuid4())
            s["source_file"] = pdf_path.name
            all_docs.append(s)

    save_json(all_docs, processed_dir / "legal_docs.json")
    return all_docs

def create_vector_store(data, persist_dir: Path):
    texts = [item["content"] for item in data]
    metadatas = [
        {"chapter": item["chapter"], "section": item["section"], "source": item["source_file"]}
        for item in data
    ]

    splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
    split_docs, split_meta = [], []

    for text, meta in zip(texts, metadatas):
        chunks = splitter.split_text(text)
        for chunk in chunks:
            split_docs.append(chunk)
            split_meta.append(meta)

    embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)

    vectordb = Chroma.from_texts(
        texts=split_docs,
        embedding=embeddings,
        metadatas=split_meta,
        persist_directory=str(persist_dir)
    )
    vectordb.persist()
    print(f"✅ Vector store created at: {persist_dir}")




In [7]:
# ==========================
# ENTRY POINT
# ==========================

if __name__ == "__main__":
    os.makedirs(PROCESSED_DIR, exist_ok=True)
    os.makedirs(VECTORSTORE_DIR, exist_ok=True)

    print("🚀 Starting Legal Document Ingestion Pipeline...")

    data = process_legal_pdfs(DATA_DIR, PROCESSED_DIR)
    if data:
        create_vector_store(data, VECTORSTORE_DIR)
        print("🎯 Ingestion + Embedding completed successfully.")
    else:
        print("⚠️ No data processed.")

🚀 Starting Legal Document Ingestion Pipeline...
⚠️ No PDF files found in /home/jagannath/my_poc_law/MyPocketLawyer-AI-Powered-Legal-Aid-Assistant/data
⚠️ No data processed.


## 🧩 Pipeline Summary

### **Step 1 — JSON Generator**

- Splits PDFs into **40-page chunks**.  
- Sends each chunk to **Gemini** for structured JSON extraction (uses gemini document understanding).  
- Merges partials and saves one JSON per document under `data/processed/`.



In [8]:
import os
import json
import re
import pathlib
import sys
from pathlib import Path
from dotenv import load_dotenv
from google import genai
from PyPDF2 import PdfReader, PdfWriter

# ============================================================
# CONFIG
# ============================================================

CHUNK_SIZE = 40  # pages per sub-PDF
PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

from config.paths import DATA_DIR, PROCESSED_DIR

DATA_DIR = DATA_DIR
OUTPUT_DIR = PROCESSED_DIR
OUTPUT_DIR.mkdir(exist_ok=True)

load_dotenv()
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

PROMPT = """
You are an AI assistant for legal document structuring and retrieval.

Task:
Extract the full *hierarchical citation structure* from the attached legal PDF chunk and return it as valid JSON.

The structure must reflect:
- Parts (with their number + title)
- Articles (number, title, clauses if any)
- Clauses or sub-clauses under each Article
- Schedules (if any)
- The preamble (if found)

JSON Schema:
{
  "document_title": str,
  "preamble": str,
  "parts": [
    {
      "part_number": str,
      "part_title": str,
      "articles": [
        {
          "article_number": str,
          "article_title": str,
          "clauses": [str]
        }
      ]
    }
  ],
  "schedules": [
    {
      "schedule_number": str,
      "schedule_title": str
    }
  ]
}

Rules:
- Output ONLY JSON. No commentary, no markdown.
- If a value doesn’t exist, use "" or [].
- Maintain exact hierarchical order.
CRITICAL: Return valid JSON or "{}" — never text commentary.
"""

# ============================================================
# UTILITIES
# ============================================================

def split_pdf(pdf_path, chunk_size=40):
    """Split a PDF into smaller temporary PDFs of `chunk_size` pages."""
    reader = PdfReader(pdf_path)
    chunks = []
    total_pages = len(reader.pages)
    for start in range(0, total_pages, chunk_size):
        end = min(start + chunk_size, total_pages)
        writer = PdfWriter()
        for i in range(start, end):
            writer.add_page(reader.pages[i])
        chunk_path = pdf_path.parent / f"__temp_{pdf_path.stem}_{start+1}-{end}.pdf"
        with open(chunk_path, "wb") as f:
            writer.write(f)
        chunks.append(chunk_path)
    return chunks

def clean_json(raw):
    """Extract valid JSON block from model output."""
    match = re.search(r"\{.*\}", raw, re.DOTALL)
    return match.group(0).strip() if match else raw.strip()

def merge_jsons(json_list):
    """Merge multiple JSON structures safely (deduplicate by part/article)."""
    if not json_list:
        return {}

    base = json_list[0].copy()
    base_parts = {p["part_number"]: p for p in base.get("parts", [])}
    base_schedules = {s["schedule_number"]: s for s in base.get("schedules", [])}

    for js in json_list[1:]:
        for part in js.get("parts", []):
            pnum = part["part_number"]
            if pnum in base_parts:
                existing_articles = {a["article_number"]: a for a in base_parts[pnum].get("articles", [])}
                for art in part.get("articles", []):
                    anum = art["article_number"]
                    if anum in existing_articles:
                        # merge clauses (dedup)
                        old_clauses = set(existing_articles[anum].get("clauses", []))
                        new_clauses = set(art.get("clauses", []))
                        existing_articles[anum]["clauses"] = list(old_clauses | new_clauses)
                    else:
                        base_parts[pnum]["articles"].append(art)
            else:
                base_parts[pnum] = part

        for sch in js.get("schedules", []):
            snum = sch["schedule_number"]
            if snum not in base_schedules:
                base_schedules[snum] = sch

    base["parts"] = list(base_parts.values())
    base["schedules"] = list(base_schedules.values())
    return base

# ============================================================
# MAIN PIPELINE
# ============================================================

pdf_paths = list(DATA_DIR.glob("*.pdf"))

for pdf_path in pdf_paths:
    print(f"\n📘 Processing: {pdf_path.name}")

    # Step 1: Split into smaller chunks
    chunk_paths = split_pdf(pdf_path, CHUNK_SIZE)
    partial_jsons = []

    # Step 2: Upload each chunk & extract JSON
    for cpath in chunk_paths:
        print(f"  ↳ Analyzing pages {cpath.name.split('_')[-1].replace('.pdf','')}")
        uploaded = client.files.upload(file=cpath.open("rb"), config={"mime_type": "application/pdf"})
        resp = client.models.generate_content(model="gemini-2.5-flash", contents=[uploaded, PROMPT])
        raw = clean_json(resp.text)

        try:
            parsed = json.loads(raw)
            if parsed:
                partial_jsons.append(parsed)
        except json.JSONDecodeError:
            # Retry once with strict "fix JSON" instruction
            retry_prompt = "Rewrite ONLY this into valid JSON according to schema:\n" + raw
            retry_resp = client.models.generate_content(model="gemini-2.5-flash", contents=[retry_prompt])
            raw_retry = clean_json(retry_resp.text)
            try:
                parsed_retry = json.loads(raw_retry)
                partial_jsons.append(parsed_retry)
            except:
                print(f"⚠️  Could not parse chunk {cpath.name}")

        # Cleanup temp chunk file
        cpath.unlink(missing_ok=True)

    # Step 3: Merge all chunk JSONs
    merged = merge_jsons(partial_jsons)

    # Step 4: Save single unified JSON per document
    out_path = OUTPUT_DIR / f"{pdf_path.stem}.json"
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(merged, f, indent=2, ensure_ascii=False)

    print(f"✅ Final JSON saved: {out_path}")


### **Step 2 — Vector Store Builder**

- Loads JSONs from `data/processed/`.  
- Flattens hierarchical structure into **granular clauses**.  
- Generates **embeddings** using Gemini.  
- Saves vector database locally under `chroma_db/`.


In [9]:
import os
import json
import sys
from pathlib import Path
from tqdm import tqdm
from dotenv import load_dotenv
import chromadb
from chromadb.config import Settings
from google import genai

# ============================================================
# CONFIG
# ============================================================

load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")
if not API_KEY:
    raise ValueError("❌ Missing GEMINI_API_KEY in .env")

client = genai.Client(api_key=API_KEY)

try:
    PROJECT_ROOT = Path(__file__).resolve().parent.parent
except NameError:
    PROJECT_ROOT = Path.cwd().parent  # when running inside a notebook

sys.path.append(str(PROJECT_ROOT))

from config.paths import DATA_DIR, PROCESSED_DIR, VECTORSTORE_DIR

DATA_DIR = DATA_DIR
OUTPUT_DIR = PROCESSED_DIR
OUTPUT_DIR.mkdir(exist_ok=True)

COLLECTION_NAME = "legal_docs"

# ============================================================
# HELPER FUNCTIONS
# ============================================================

def load_json_files(folder):
    """Load all JSON files from folder."""
    json_files = list(Path(folder).glob("*.json"))
    data = []
    for f in json_files:
        try:
            with open(f, "r", encoding="utf-8") as jf:
                data.append((f.stem, json.load(jf)))
        except Exception as e:
            print(f"⚠️ Failed to load {f.name}: {e}")
    return data


def flatten_legal_json(doc_title, js):
    """
    Flatten legal JSON:
    Each clause becomes an entry with metadata for part, article, and source document.
    """
    entries = []

    # Preamble (optional)
    if js.get("preamble"):
        entries.append({
            "text": js["preamble"].strip(),
            "metadata": {
                "document_title": doc_title,
                "section": "Preamble"
            }
        })

    # Parts and Articles
    for part in js.get("parts", []):
        pnum = part.get("part_number", "")
        ptitle = part.get("part_title", "")
        for article in part.get("articles", []):
            anum = article.get("article_number", "")
            atitle = article.get("article_title", "")
            clauses = article.get("clauses", [])

            # Each clause stored separately for fine-grained retrieval
            for idx, clause in enumerate(clauses, start=1):
                entries.append({
                    "text": clause.strip(),
                    "metadata": {
                        "document_title": doc_title,
                        "part_number": pnum,
                        "part_title": ptitle,
                        "article_number": anum,
                        "article_title": atitle,
                        "clause_index": idx,
                        "section": "Clause"
                    }
                })
    return entries


def get_embeddings(texts):
    """Generate embeddings using Gemini."""
    response = client.models.embed_content(
        model="models/text-embedding-004",
        contents=texts
    )
    return [e.values for e in response.embeddings]


# ============================================================
# MAIN PIPELINE
# ============================================================

def build_chroma_store():
    print("📚 Loading processed JSONs...")
    data = load_json_files(PROCESSED_DIR)

    all_entries = []
    for doc_name, js in tqdm(data, desc="Flattening documents"):
        all_entries.extend(flatten_legal_json(doc_name, js))

    print(f"🧩 Total {len(all_entries)} text entries to embed")

    client_chroma = chromadb.PersistentClient(path=str(VECTORSTORE_DIR))

    # Remove old collection if exists
    if COLLECTION_NAME in [c.name for c in client_chroma.list_collections()]:
        client_chroma.delete_collection(COLLECTION_NAME)

    collection = client_chroma.create_collection(name=COLLECTION_NAME)


    BATCH_SIZE = 50
    for i in tqdm(range(0, len(all_entries), BATCH_SIZE), desc="Embedding & storing"):
        batch = all_entries[i:i + BATCH_SIZE]
        texts = [e["text"] for e in batch]
        metadatas = [e["metadata"] for e in batch]
        ids = [f"doc_{i+j}" for j in range(len(batch))]

        embeddings = get_embeddings(texts)
        collection.add(
            ids=ids,
            embeddings=embeddings,
            documents=texts,
            metadatas=metadatas
        )

    print(f"✅ Vectorstore created at: {VECTORSTORE_DIR}")


if __name__ == "__main__":
    build_chroma_store()


📚 Loading processed JSONs...


Flattening documents: 100%|██████████| 5/5 [00:00<00:00, 313.99it/s]

🧩 Total 5037 text entries to embed



Embedding & storing: 100%|██████████| 101/101 [02:58<00:00,  1.77s/it]

✅ Vectorstore created at: /home/jagannath/my_poc_law/MyPocketLawyer-AI-Powered-Legal-Aid-Assistant/chroma_db
